In [2]:
import pandas as pd

In [48]:
df = pd.read_csv('reviews.csv')

In [5]:
df.head()

,genre,name,critic_score,audience_score,critic_reviews,audience_reviews,critic_sentiment_result,audience_sentiment_result,critic_postive,critic_negative,critic_neutral,critic_mixed,critic_sentiment,audience_postive,audience_negative,audience_neutral,audience_mixed,audience_sentiment
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,0.231601,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE
1,Drama,the_taste_of_things,98,100,Although its delectable dishes provide a feast...,"This movie is both complex and simple, like a ...","{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.006401,0.006116,0.001319,0.986164,MIXED,0.994632,0.001496,0.003765,0.000107,POSITIVE
2,Drama,killers_of_the_flower_moon,93,84,"The development of the characters, the perform...",so slow and the plot is terrible. I agree the ...,"{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.278133,0.004101,0.017245,0.700520,MIXED,0.003707,0.721609,0.001737,0.272947,NEGATIVE
3,Drama,our_son,89,62,"As a director, Bill Oliver never allows this t...",Totally hot cast! Really heartwarming story an...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.975298,0.003327,0.007768,0.013607,POSITIVE,0.999808,0.000053,0.000112,0.000026,POSITIVE
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,0.143244,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE


In [9]:
df['audience_reviews'][2]

"so slow and the plot is terrible. I agree the acting is fine and the directing is good just hated the story. Good story, good acting, but too long and too transparent. herein is over 3 hours of my journey... somewhat ironically, it definitely presents the challenging journey of others, in nearly slow motion... this likely will feel particularly uncomfortable. i respectfully watched everything produced, in its entirety. please find your own opinion. if you're still reading this you might agree to the failure of editing. assuredly, make certain you have have caffeine or alternative, as this feature could induce dozing. An instant classic. Scorsese perfected the movie formula a while ago and this is just another incredible example. If you are up for some slow pace but good performance from the leads its the movie for you. Scorsese missed so many important facts for this story.  Location of the Osage Agency, the BIA offices, plus the courthouse were ALL in Pawhuska, not Fairfax.     These

In [32]:
import pprint
from pandas import json_normalize
import boto3

pp = pprint.PrettyPrinter(indent=2)
# %%
# Language Detection
comprehend = boto3.client(service_name="comprehend", region_name="eu-west-1")

In [45]:
def run_targeted_sentiment_analysis(text):
    sentiment_result = comprehend.detect_targeted_sentiment(Text=df['audience_reviews'][2], LanguageCode="en")
    return sentiment_result

In [24]:
targeted_sentiment = comprehend.detect_targeted_sentiment(Text=df['audience_reviews'][2], LanguageCode="en")

In [25]:
targeted_sentiment['Entities'][0]['Mentions'][0]['Text']

'plot'

In [26]:
targeted_sentiment['Entities'][0]['Mentions'][0]['MentionSentiment']['Sentiment']

'NEGATIVE'

In [27]:
targeted_sentiment_df = pd.DataFrame([targeted_sentiment])
targeted_sentiment_df['targeted_text'] = targeted_sentiment_df['Entities'].apply(lambda x: x[1]['Mentions'][0]['Text'])
targeted_sentiment_df['targeted_sentiment'] = targeted_sentiment_df['Entities'].apply(lambda x: x[0]['Mentions'][0]['MentionSentiment']['Sentiment'])


In [30]:
targeted_sentiment_df

,Entities,ResponseMetadata,targeted_text,targeted_sentiment
0,"[{'DescriptiveMentionIndex': [0], 'Mentions': ...",{'RequestId': '1724d7fb-7328-4d35-9388-624105b...,my,NEUTRAL


In [31]:
targeted_sentiment_df['targeted_text'] = targeted_sentiment_df['Entities'].apply(lambda x: x[1]['Mentions'][0]['Text'])
targeted_sentiment_df['targeted_sentiment'] = targeted_sentiment_df['Entities'].apply(lambda x: x[1]['Mentions'][0]['MentionSentiment']['Sentiment'])

In [61]:
targeted_sentiment = run_targeted_sentiment_analysis(df['audience_reviews'][2])

In [65]:
targeted_sentiment['Entities'][0]['Mentions']

[{'Score': 0.9557300209999084,
  'GroupScore': 1.0,
  'Text': 'plot',
  'Type': 'ATTRIBUTE',
  'MentionSentiment': {'Sentiment': 'NEGATIVE',
   'SentimentScore': {'Positive': 0.0,
    'Negative': 0.9999979734420776,
    'Neutral': 9.999999974752427e-07,
    'Mixed': 9.999999974752427e-07}},
  'BeginOffset': 16,
  'EndOffset': 20}]

In [123]:
def extract_all_entities(json_response):
    entities_list = json_response['Entities'] if 'Entities' in json_response else []
    text_list = []
    sentiment_list = []

    for entity in entities_list:
        mentions_list = entity['Mentions'] if 'Mentions' in entity else []
        
        if mentions_list:
            mention = mentions_list[0]
            text = mention['Text'] if 'Text' in mention else None
            sentiment = mention['MentionSentiment']['Sentiment'] if 'MentionSentiment' in mention else None
            
            text_list.append(text)
            sentiment_list.append(sentiment)

    result = {'Text': text_list, 'Sentiment': sentiment_list}
    return result


In [124]:
result = extract_all_entities(targeted_sentiment)
result_df = pd.DataFrame(result)
result_df

,Text,Sentiment
0,plot,NEGATIVE
1,my,NEUTRAL
2,directing,POSITIVE
3,story,POSITIVE
4,acting,POSITIVE
5,over 3,NEUTRAL
6,others,NEUTRAL
7,your,NEUTRAL
8,you,NEUTRAL
9,you,NEUTRAL


In [100]:
extract_all_entities(test_df['targeted_sentiment'][0])

{'Text': [], 'Sentiment': []}

In [56]:
def get_targeted_sentiment(text):
    targeted_sentiment = run_targeted_sentiment_analysis(text)
    texts = []
    sentiments = []
    for i in range(len(targeted_sentiment['Entities'])):
        text = targeted_sentiment['Mentions'][i][0]['Text']
        sentiment = targeted_sentiment['Mentions'][i][0]['MentionSentiment']['Sentiment']
        texts.append(text)
        sentiments.append(sentiment)
    targeted_dict = {'text':texts, 'sentiment':sentiments}
    return targeted_dict

In [ ]:
normalized_df = json_normalize(targeted_sentiment['Entities'], sep='_')
texts = []
sentiments = []
for i in range(len(targeted_sentiment['Entities'])):
    text = normalized_df['Mentions'][i][0]['Text']
    sentiment = normalized_df['Mentions'][i][0]['MentionSentiment']['Sentiment']
    texts.append(text)
    sentiments.append(sentiment)
targeted_dict = {'text':texts, 'sentiment':sentiments}

In [125]:
test_df = df.head()

In [126]:
test_df

,genre,name,critic_score,audience_score,critic_reviews,audience_reviews,critic_sentiment_result,audience_sentiment_result,critic_postive,critic_negative,critic_neutral,critic_mixed,critic_sentiment,audience_postive,audience_negative,audience_neutral,audience_mixed,audience_sentiment
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,0.231601,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE
1,Drama,the_taste_of_things,98,100,Although its delectable dishes provide a feast...,"This movie is both complex and simple, like a ...","{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.006401,0.006116,0.001319,0.986164,MIXED,0.994632,0.001496,0.003765,0.000107,POSITIVE
2,Drama,killers_of_the_flower_moon,93,84,"The development of the characters, the perform...",so slow and the plot is terrible. I agree the ...,"{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.278133,0.004101,0.017245,0.700520,MIXED,0.003707,0.721609,0.001737,0.272947,NEGATIVE
3,Drama,our_son,89,62,"As a director, Bill Oliver never allows this t...",Totally hot cast! Really heartwarming story an...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.975298,0.003327,0.007768,0.013607,POSITIVE,0.999808,0.000053,0.000112,0.000026,POSITIVE
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,0.143244,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE


In [127]:
test_df['targeted_sentiment'] = test_df['audience_reviews'].apply(lambda x: run_targeted_sentiment_analysis(x))


/tmp/ipykernel_1731/4274963740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['targeted_sentiment'] = test_df['audience_reviews'].apply(lambda x: run_targeted_sentiment_analysis(x))


In [128]:
test_df['targeted_sentiment'] = test_df['targeted_sentiment'].apply(lambda x: extract_all_entities(x))

/tmp/ipykernel_1731/1193023813.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['targeted_sentiment'] = test_df['targeted_sentiment'].apply(lambda x: extract_all_entities(x))


In [129]:
test_df

,genre,name,critic_score,audience_score,critic_reviews,audience_reviews,critic_sentiment_result,audience_sentiment_result,critic_postive,critic_negative,critic_neutral,critic_mixed,critic_sentiment,audience_postive,audience_negative,audience_neutral,audience_mixed,audience_sentiment,targeted_sentiment
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,0.231601,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
1,Drama,the_taste_of_things,98,100,Although its delectable dishes provide a feast...,"This movie is both complex and simple, like a ...","{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.006401,0.006116,0.001319,0.986164,MIXED,0.994632,0.001496,0.003765,0.000107,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
2,Drama,killers_of_the_flower_moon,93,84,"The development of the characters, the perform...",so slow and the plot is terrible. I agree the ...,"{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.278133,0.004101,0.017245,0.700520,MIXED,0.003707,0.721609,0.001737,0.272947,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
3,Drama,our_son,89,62,"As a director, Bill Oliver never allows this t...",Totally hot cast! Really heartwarming story an...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.975298,0.003327,0.007768,0.013607,POSITIVE,0.999808,0.000053,0.000112,0.000026,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,0.143244,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."


In [130]:
test_df

,genre,name,critic_score,audience_score,critic_reviews,audience_reviews,critic_sentiment_result,audience_sentiment_result,critic_postive,critic_negative,critic_neutral,critic_mixed,critic_sentiment,audience_postive,audience_negative,audience_neutral,audience_mixed,audience_sentiment,targeted_sentiment
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,0.231601,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
1,Drama,the_taste_of_things,98,100,Although its delectable dishes provide a feast...,"This movie is both complex and simple, like a ...","{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.006401,0.006116,0.001319,0.986164,MIXED,0.994632,0.001496,0.003765,0.000107,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
2,Drama,killers_of_the_flower_moon,93,84,"The development of the characters, the perform...",so slow and the plot is terrible. I agree the ...,"{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.278133,0.004101,0.017245,0.700520,MIXED,0.003707,0.721609,0.001737,0.272947,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
3,Drama,our_son,89,62,"As a director, Bill Oliver never allows this t...",Totally hot cast! Really heartwarming story an...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.975298,0.003327,0.007768,0.013607,POSITIVE,0.999808,0.000053,0.000112,0.000026,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,0.143244,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ..."


In [131]:
test_df['text'] = test_df['targeted_sentiment'].apply(lambda x: x['Text'])
test_df['sentiment'] = test_df['targeted_sentiment'].apply(lambda x: x['Sentiment'])

/tmp/ipykernel_1731/1536222976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['text'] = test_df['targeted_sentiment'].apply(lambda x: x['Text'])
/tmp/ipykernel_1731/1536222976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['sentiment'] = test_df['targeted_sentiment'].apply(lambda x: x['Sentiment'])


In [132]:
test_df

,genre,name,critic_score,audience_score,critic_reviews,audience_reviews,critic_sentiment_result,audience_sentiment_result,critic_postive,critic_negative,...,critic_mixed,critic_sentiment,audience_postive,audience_negative,audience_neutral,audience_mixed,audience_sentiment,targeted_sentiment,text,sentiment
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,...,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...","[plot, my, directing, story, acting, over 3, o...","[NEGATIVE, NEUTRAL, POSITIVE, POSITIVE, POSITI..."
1,Drama,the_taste_of_things,98,100,Although its delectable dishes provide a feast...,"This movie is both complex and simple, like a ...","{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.006401,0.006116,...,0.986164,MIXED,0.994632,0.001496,0.003765,0.000107,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...","[plot, my, directing, story, acting, over 3, o...","[NEGATIVE, NEUTRAL, POSITIVE, POSITIVE, POSITI..."
2,Drama,killers_of_the_flower_moon,93,84,"The development of the characters, the perform...",so slow and the plot is terrible. I agree the ...,"{'Sentiment': 'MIXED', 'SentimentScore': {'Pos...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.278133,0.004101,...,0.700520,MIXED,0.003707,0.721609,0.001737,0.272947,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...","[plot, my, directing, story, acting, over 3, o...","[NEGATIVE, NEUTRAL, POSITIVE, POSITIVE, POSITI..."
3,Drama,our_son,89,62,"As a director, Bill Oliver never allows this t...",Totally hot cast! Really heartwarming story an...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.975298,0.003327,...,0.013607,POSITIVE,0.999808,0.000053,0.000112,0.000026,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...","[plot, my, directing, story, acting, over 3, o...","[NEGATIVE, NEUTRAL, POSITIVE, POSITIVE, POSITI..."
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,...,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...","[plot, my, directing, story, acting, over 3, o...","[NEGATIVE, NEUTRAL, POSITIVE, POSITIVE, POSITI..."


In [133]:
long_table = test_df.explode('text').explode('sentiment')


In [134]:
long_table

,genre,name,critic_score,audience_score,critic_reviews,audience_reviews,critic_sentiment_result,audience_sentiment_result,critic_postive,critic_negative,...,critic_mixed,critic_sentiment,audience_postive,audience_negative,audience_neutral,audience_mixed,audience_sentiment,targeted_sentiment,text,sentiment
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,...,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",plot,NEGATIVE
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,...,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",plot,NEUTRAL
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,...,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",plot,POSITIVE
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,...,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",plot,POSITIVE
0,Drama,four_daughters_2023,96,91,In telling the story of Olfa and her daughters...,The most compelling documentary of 2023. Stunn...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'POSITIVE', 'SentimentScore': {'...",0.765325,0.001325,...,0.001748,POSITIVE,0.999406,0.000024,0.000545,0.000025,POSITIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",plot,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,...,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",I,NEUTRAL
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,...,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",I,POSITIVE
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,...,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",I,NEUTRAL
4,Drama,priscilla,82,63,"Of all the directors working today, there is n...",I read the book and was really excited for thi...,"{'Sentiment': 'POSITIVE', 'SentimentScore': {'...","{'Sentiment': 'NEGATIVE', 'SentimentScore': {'...",0.845207,0.002500,...,0.009050,POSITIVE,0.001825,0.935506,0.000316,0.062354,NEGATIVE,"{'Text': ['plot', 'my', 'directing', 'story', ...",I,NEGATIVE


In [ ]:
# Apply sentiment analysis and extract required information
sent_df['critic_sentiment_result'] = sent_df['critic_reviews'].apply(run_sentiment_analysis)
sent_df['audience_sentiment_result'] = sent_df['audience_reviews'].apply(run_sentiment_analysis)

# Extract positive, negative, neutral, mixed, and overall sentiment
sent_df['critic_postive'] = sent_df['critic_sentiment_result'].apply(lambda x: x['SentimentScore']['Positive'])
sent_df['critic_negative'] = sent_df['critic_sentiment_result'].apply(lambda x: x['SentimentScore']['Negative'])
sent_df['critic_neutral'] = sent_df['critic_sentiment_result'].apply(lambda x: x['SentimentScore']['Neutral'])
sent_df['critic_mixed'] = sent_df['critic_sentiment_result'].apply(lambda x: x['SentimentScore']['Mixed'])
sent_df['critic_sentiment'] = sent_df['critic_sentiment_result'].apply(lambda x: x['Sentiment'])

sent_df['audience_postive'] = sent_df['audience_sentiment_result'].apply(lambda x: x['SentimentScore']['Positive'])
sent_df['audience_negative'] = sent_df['audience_sentiment_result'].apply(lambda x: x['SentimentScore']['Negative'])
sent_df['audience_neutral'] = sent_df['audience_sentiment_result'].apply(lambda x: x['SentimentScore']['Neutral'])
sent_df['audience_mixed'] = sent_df['audience_sentiment_result'].apply(lambda x: x['SentimentScore']['Mixed'])
sent_df['audience_sentiment'] = sent_df['audience_sentiment_result'].apply(lambda x: x['Sentiment'])
